In [13]:
from idmanagerclient import IDManager
from lodgatewayclient import LODGatewayClient
import json

prod_idm = IDManager("https://services.getty.edu/id-management")
rcv = LODGatewayClient("https://data.getty.edu/research/collections")

# pretty print
def pp(jsondoc):
    print(json.dumps(jsondoc, indent=2, sort_keys=True))

In [14]:
print(f"Connected to Production IDM (readonly): {prod_idm.ready}")
print(f"Connected to Production RCV (readonly): {rcv.ready}")

Connected to Production IDM (readonly): True
Connected to Production RCV (readonly): True


In [15]:
# Get a list of collection URIs from IDM (collections have accession numbers)
collections = [link['body']['id'] for link in prod_idm.query_links(body_generator="lodlvl2", motivation="contains-accno")]
collections[0]


'https://data.getty.edu/research/collections/component/0f9fa712-f746-5341-96b4-538a8efbe85b'

In [16]:
collection_jsonld = rcv.get_item(collections[0])
print(collection_jsonld['_label'])

Getty Research Institute public event records and recordings


In [17]:
subjects = collection_jsonld['about']
pp(subjects[:4])

[
  {
    "_label": "Artists -- California",
    "id": "https://data.getty.edu/research/collections/document/fe0bb3bf-2d13-5c00-be2b-db3bd01f41a5",
    "type": "LinguisticObject"
  },
  {
    "_label": "Minimal art",
    "id": "https://data.getty.edu/research/collections/document/ef1d3527-b87d-5b63-9be2-704b871dbabd",
    "type": "LinguisticObject"
  },
  {
    "_label": "Video art",
    "id": "https://data.getty.edu/research/collections/document/bbbbb263-a129-5f4b-a181-b9be9fb1a8ad",
    "type": "LinguisticObject"
  },
  {
    "_label": "Art, Modern -- 20th century",
    "id": "https://data.getty.edu/research/collections/document/1971f690-bf90-5e9d-9f6a-256ff3d9cfee",
    "type": "LinguisticObject"
  }
]


In [18]:
subject = rcv.get_item(subjects[0]['id'])
pp(subject)

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "classified_as": [
    {
      "_label": "Subject Heading - Topical",
      "id": "https://data.getty.edu/local/thesaurus/aspace-subject-topical",
      "type": "Type"
    }
  ],
  "content": "Artists -- California",
  "format": "text/html",
  "id": "https://data.getty.edu/research/collections/document/fe0bb3bf-2d13-5c00-be2b-db3bd01f41a5",
  "identified_by": [
    {
      "classified_as": [
        {
          "_label": "generated URL slug",
          "id": "https://data.getty.edu/local/thesaurus/temporary-slug",
          "type": "Type"
        },
        {
          "_label": "Slug Identifier",
          "id": "https://data.getty.edu/local/thesaurus/slug-identifier",
          "type": "Type"
        }
      ],
      "content": "10PEP3",
      "id": "https://data.getty.edu/research/collections/document/fe0bb3bf-2d13-5c00-be2b-db3bd01f41a5/slug",
      "type": "Identifier"
    }
  ],
  "part": [
    {
      "id": "https://d

In [21]:
# What resources does this subject get applied to?
results = rcv.sparql("select ?g where { GRAPH ?g {?s ?p <" + subject['id'] + "> .}} LIMIT 100")

# Get the label of each thing and print it
# (Deal with the SPARQL SELECT JSON response:)
for r in results['results']['bindings']:
    uri = r['g']['value']
    resource = rcv.get_item(uri)
    print(f"{uri} - {resource.get('_label')}")

https://data.getty.edu/research/collections/component/3eb56258-f740-56db-8e62-45b9846c9b53 - Sam Francis papers
https://data.getty.edu/research/collections/component/0f9fa712-f746-5341-96b4-538a8efbe85b - Getty Research Institute public event records and recordings
https://data.getty.edu/research/collections/component/6e938320-c4e6-5821-aaf1-18a95845d286 - Barbara T. Smith papers
https://data.getty.edu/research/collections/component/0302c3e8-b951-527c-87f8-3ba8e2d0cd3e - Rolf Nelson Gallery records
https://data.getty.edu/research/collections/component/0c934541-1330-5857-aaf3-62f95a09ab8d - Hal Glicksman papers
https://data.getty.edu/research/collections/component/6f316612-d39f-54d9-a7f0-3a155a5eae6e - Jan Baum Gallery records
https://data.getty.edu/research/collections/component/678c8f47-0e05-56eb-bc81-3b86d0ec0e77 - Getty Research Institute California Video exhibition project files and oral history recordings
